In [1]:
import pandas as pd
import numpy as np
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
import urllib.request
import zipfile
import os
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,LSTM,GRU,Dense
import nltk
from nltk.tokenize import word_tokenize
import warnings
import tensorflow as tf
nltk.download('punkt')
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /home/tham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
csv_table=pd.read_table('/media/tham/SSD500/Workspace/challenge/train.tsv',sep='\t')
csv_table.to_csv('/media/tham/SSD500/Workspace/challenge/train.csv',index=False)
train_df = pd.read_csv("/media/tham/SSD500/Workspace/challenge/train.csv")
# train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=2020)
# train_df.to_csv('/media/tham/SSD500/Workspace/challenge/train_80.csv', index=False)
val_df=pd.read_csv("/media/tham/SSD500/Workspace/challenge/dev_with_labels.csv")

In [3]:
train_df = pd.read_csv("/media/tham/SSD500/Workspace/challenge/train.csv")
x_train =train_df['Text_data']
y_train =train_df['Label']
test_df = pd.read_csv("/media/tham/SSD500/Workspace/challenge/dev_with_labels.csv")
x_test=test_df['Text data']
y_test=test_df['Label']

In [4]:
data_train=pd.DataFrame({'Text':x_train,'Emotion':y_train})
data_test=pd.DataFrame({'Text':x_test,'Emotion':y_test})
data=data_train.append(data_test,ignore_index=True)

In [5]:
def clean_text(data):
    data=re.sub(r"[^a-zA-Z\d]", " ", data)
    data=re.sub(r"(@[\d\w\.]+)", '', data)
    data=word_tokenize(data)
    return data

texts=[' '.join(clean_text(text)) for text in data.Text]
texts_train=[' '.join(clean_text(text)) for text in x_train]
texts_test=[' '.join(clean_text(text)) for text in x_test]

In [6]:
len(data_train)

8891

In [7]:

# data=re.sub(r"(#[\d\w\.]+)", '', str(data))

In [8]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(texts)
sequence_train=tokenizer.texts_to_sequences(texts_train)
sequence_test=tokenizer.texts_to_sequences(texts_test)
index_of_words=tokenizer.word_index
vocab_size=len(index_of_words)+1

In [9]:
num_classes=3
embed_num_dims=300
max_seq_len=500
class_names=['moderate','not depression','severe']
X_train_pad=pad_sequences(sequence_train,maxlen=max_seq_len)
X_test_pad=pad_sequences(sequence_test,maxlen=max_seq_len)
encoding={'moderate':0,'not depression':1,'severe':2}
y_train=[encoding[x] for x in data_train.Emotion]
y_test=[encoding[x] for x in data_test.Emotion]
y_train=to_categorical(y_train)
y_test=to_categorical(y_test) 

In [10]:
def create_embedding_matrix(filepath,word_index,embedding_dim):
    vocab_size=len(word_index)+1
    embedding_matrix=np.zeros((vocab_size,embedding_dim))
    with open(filepath) as f:
        for line in f:
            word,*vector=line.split()
            if word in word_index:
                idx=word_index[word]
                embedding_matrix[idx] = np.array(vector,dtype=np.float32)[:embedding_dim]
    return embedding_matrix
fname='/media/tham/SSD500/Workspace/challenge/Emotion/wiki-news-300d-1M.vec'
embedd_matrix=create_embedding_matrix(fname,index_of_words,embed_num_dims)

In [11]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
embedd_layer=Embedding(vocab_size,embed_num_dims,input_length=max_seq_len,weights=[embedd_matrix],trainable=False)
gru_output_size=128
bidirectional=True
model=Sequential()
model.add(embedd_layer)
model.add(Bidirectional(GRU(units=gru_output_size,dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
batch_size=128
epochs=10
hist=model.fit(X_train_pad,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test_pad,y_test))

Epoch 1/10
70/70 [==============================] - 121s 2s/step - loss: 0.8092 - accuracy: 0.6756 - val_loss: 0.9663 - val_accuracy: 0.5592
Epoch 2/10
70/70 [==============================] - 118s 2s/step - loss: 0.7032 - accuracy: 0.7273 - val_loss: 1.0648 - val_accuracy: 0.5323
Epoch 3/10
70/70 [==============================] - 119s 2s/step - loss: 0.6366 - accuracy: 0.7581 - val_loss: 1.2170 - val_accuracy: 0.5601
Epoch 4/10
70/70 [==============================] - 117s 2s/step - loss: 0.5597 - accuracy: 0.7937 - val_loss: 1.0977 - val_accuracy: 0.5623
Epoch 5/10
70/70 [==============================] - 122s 2s/step - loss: 0.5171 - accuracy: 0.8127 - val_loss: 1.1654 - val_accuracy: 0.5467
Epoch 6/10
70/70 [==============================] - 121s 2s/step - loss: 0.4817 - accuracy: 0.8328 - val_loss: 1.2032 - val_accuracy: 0.5351
Epoch 7/10
70/70 [==============================] - 119s 2s/step - loss: 0.4321 - accuracy: 0.8510 - val_loss: 1.3143 - val_accuracy: 0.5211
Epoch 8/10
70